In [6]:
import json
with open("sotopia_pi_bc_episodes_reward_direct_5-scale_gpt-4o.json", 'r') as f:
    direct_5 = json.load(f)
with open("sotopia_pi_bc_episodes_reward_direct_default_o3-mini.json", 'r') as f:
    direct_o3 = json.load(f)
with open("sotopia_pi_bc_episodes_reward_direct_average_gpt-4o.json", 'r') as f:
    direct_avg = json.load(f)
with open("sotopia_pi_bc_episodes_reward_discounting.json", 'r') as f:
    discounting = json.load(f)
with open("sotopia_pi_bc_episodes_reward_goal_progress_gpt-4o.json", 'r') as f:
    goal_progress = json.load(f)
with open("sotopia_pi_bc_episodes_reward_only_response_gpt-4o.json", 'r') as f:
    only_response = json.load(f)
with open("sotopia_pi_bc_episodes_reward_utterance_quality_gpt-4o.json", 'r') as f:
    utterance_quality = json.load(f)

from collections import Counter

In [9]:
counter_direct_5 = Counter([dic["value"] for dic in direct_5])
counter_direct_o3 = Counter([dic["value"] for dic in direct_o3])
counter_direct_avg = Counter([dic["value"] for dic in direct_avg])
counter_discounting = Counter([dic["value"] for dic in discounting])
counter_goal_progress = Counter([dic["value"] for dic in goal_progress])
counter_only_response = Counter([dic["value"] for dic in only_response])
counter_utterance_quality = Counter([dic["value"] for dic in utterance_quality])

In [17]:
def plot_6_buckets(counter, title):
    # plot 6 buckets: -inf to 0, 0 to 2, 2 to 4, 4 to 6, 6 to 8, 8 to inf
    print(title)
    print("bucket\tcount")
    count = sum([v for k, v in counter.items() if k < 0])
    print("-inf to 0\t", count)
    for i in range(5):
        count = sum([v for k, v in counter.items() if i*2 <= k < (i+1)*2])
        print(f"{i*2} to {(i+1)*2}\t{count}")
    count = sum([v for k, v in counter.items() if k >= 10])
    print()
    print(sum([v for k, v in counter.items()]), "total")
    
plot_6_buckets(counter_direct_5, "direct_5")
plot_6_buckets(counter_direct_o3, "direct_o3")
plot_6_buckets(counter_direct_avg, "direct_avg")
plot_6_buckets(counter_discounting, "discounting")
plot_6_buckets(counter_goal_progress, "goal_progress")
plot_6_buckets(counter_only_response, "only_response")
plot_6_buckets(counter_utterance_quality, "utterance_quality")

NameError: name 'counter_direct_5' is not defined

In [27]:
def get_hash(input, output):
    return f"{input}->{output}"

goal_progress_hash = set()
for goal_progress_dic in goal_progress:
    goal_progress_dic["hash"] = get_hash(goal_progress_dic["input"], goal_progress_dic["output"])
    goal_progress_hash.add(goal_progress_dic["hash"])
print(len(goal_progress_hash))

only_response_hash = set()
for only_response_dic in only_response:
    only_response_dic["hash"] = get_hash(only_response_dic["input"], only_response_dic["output"])
    only_response_hash.add(only_response_dic["hash"])
print(len(only_response_hash))

print(len(goal_progress_hash & only_response_hash))

7557
7568
7557


In [28]:
goal_progress_dict = {dic["hash"]: dic for dic in goal_progress}
only_response_dict = {dic["hash"]: dic for dic in only_response}

In [29]:
weights = [0.5, 0.5]
mixed = []
for goal_progress_record in goal_progress:
    hash_key = get_hash(goal_progress_record["input"], goal_progress_record["output"])
    only_response_record = only_response_dict[hash_key]
    mixed.append({
        "input": goal_progress_record["input"],
        "output": goal_progress_record["output"],
        "value": goal_progress_record["value"] * weights[0] + only_response_record["value"] * weights[1]
    })
plot_6_buckets(Counter([dic["value"] for dic in mixed]), "mixed")

mixed
bucket	count
-inf to 0	 85
0 to 2	753
2 to 4	1222
4 to 6	3655
6 to 8	1339
8 to 10	444

7557 total


In [24]:
with open("sotopia_pi_bc_episodes_reward_mixed.json", 'w') as f:
    json.dump(mixed, f, indent=2)

In [32]:
def get_hash(input, output):
    return f"{input}->{output}"

direct_o3_hash = set()
for direct_o3_dic in direct_o3:
    direct_o3_dic["hash"] = get_hash(direct_o3_dic["input"], direct_o3_dic["output"])
    direct_o3_hash.add(direct_o3_dic["hash"])
print(len(direct_o3_hash))

only_response_hash = set()
for only_response_dic in only_response:
    only_response_dic["hash"] = get_hash(only_response_dic["input"], only_response_dic["output"])
    only_response_hash.add(only_response_dic["hash"])
print(len(only_response_hash))

print(len(direct_o3_hash & only_response_hash))

7568
7568
7568


In [33]:
direct_o3_dict = {dic["hash"]: dic for dic in direct_o3}
only_response_dict = {dic["hash"]: dic for dic in only_response}

In [34]:
weights = [0.5, 0.5]
mixed = []
for direct_o3_record in direct_o3:
    hash_key = get_hash(direct_o3_record["input"], direct_o3_record["output"])
    only_response_record = only_response_dict[hash_key]
    mixed.append({
        "input": direct_o3_record["input"],
        "output": direct_o3_record["output"],
        "value": direct_o3_record["value"] * weights[0] + only_response_record["value"] * weights[1]
    })
plot_6_buckets(Counter([dic["value"] for dic in mixed]), "mixed")

mixed
bucket	count
-inf to 0	 3
0 to 2	714
2 to 4	537
4 to 6	1560
6 to 8	2788
8 to 10	1247

7575 total


In [35]:
with open("sotopia_pi_bc_episodes_reward_mixed_direct_o3_only_response.json", 'w') as f:
    json.dump(mixed, f, indent=2)

In [11]:
import json
with open("sotopia_pi_bc_episodes_reward_direct_default_gpt-4o.json", 'r') as f:
    data = json.load(f)

In [22]:
new_data = []
neg_count = 0
for d in data:
    if d["value"] < 0:
        neg_count += 1
    new_d = {"input": d["input"], "output": d["output"], "value": d["value"] if d["value"] >= 0 else 0}
    new_data.append(new_d)
print(neg_count)

9


In [23]:
from collections import Counter

counter_direct = Counter([dic["value"] for dic in new_data])
plot_6_buckets(Counter(counter_direct), "mixed")

mixed
bucket	count
-inf to 0	 0
0 to 2	1961
2 to 4	2463
4 to 6	381
6 to 8	1583
8 to 10	214

7568 total


In [25]:
with open("sotopia_pi_bc_episodes_reward_direct_default_gpt-4o_non_neg.json", 'w') as f:
    json.dump(new_data, f, indent=4)